In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import mask_rcnn
import mask_rcnn_utils
import eye_segmentation
import cv2
from tqdm import tqdm

import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM
from tensorflow.keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


STEPS PER EPOCH:  8916
VALIDATION STEPS:  2403


In [2]:
# Train
model = eye_segmentation.train_mrcnn(load_last=True) # change to False if running for first time, then change back

Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________

/home/naproxa/miniconda3/envs/biods220-env2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/naproxa/miniconda3/envs/biods220-env2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/naproxa/miniconda3/envs/biods220-env2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train all layers

Starting at epoch 11. LR=0.001

Checkpoint Path: logs/eye_segmentation20200303T0134/mask_rcnn_eye_segmentation_{epoch:04d}.h5
In model:  rpn_model


/home/naproxa/miniconda3/envs/biods220-env2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/naproxa/miniconda3/envs/biods220-env2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/naproxa/miniconda3/envs/biods220-env2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________

In [3]:
# segmentation dataset
DATASET_DIR = '/home/naproxa/cs271proj/Semantic_Segmentation_Dataset' # replace with your path
TRAIN_IMAGE_DIR = DATASET_DIR + '/train/images'
TRAIN_LABELS_DIR = DATASET_DIR + '/train/labels'
VAL_IMAGE_DIR = DATASET_DIR + '/validation/images'
VAL_LABELS_DIR = DATASET_DIR + '/validation/labels'
TEST_IMAGE_DIR = DATASET_DIR + '/test/images'

np.set_printoptions(threshold=sys.maxsize)

In [5]:
def load_data(IMAGE_DIR, LABEL_DIR):

    images = []
    image_list = sorted(os.listdir(IMAGE_DIR))
    if LABEL_DIR:
        labels = []
        label_list = sorted(os.listdir(LABEL_DIR))

    for i in tqdm(range(len(image_list))):

        image_path = IMAGE_DIR +'/'+ image_list[i]
        
        # image is duplicated across channels, take 1st channel
        img = img_to_array(load_img(image_path), dtype=np.uint8)
        images.append(img[:, :, 0])
        
        if LABEL_DIR:
            label_path = LABEL_DIR +'/'+ label_list[i]
            label = np.load(label_path)
            labels.append(label)

    if LABEL_DIR:
        return np.array(images), np.array(labels)
    else:
        return np.array(images)

train_images, train_labels = load_data(TRAIN_IMAGE_DIR, TRAIN_LABELS_DIR)
val_images, val_labels = load_data(VAL_IMAGE_DIR, VAL_LABELS_DIR)
test_images = load_data(TEST_IMAGE_DIR, None)

print('Train images: ', train_images.shape)
print('Train labels: ', train_labels.shape)
print('Val images: ', val_images.shape)
print('Val labels: ', val_labels.shape)
print('Test images: ', test_images.shape)

100%|██████████| 1440/1440 [00:38<00:00, 36.94it/s]


Train images:  (8916, 640, 400)
Train labels:  (8916, 640, 400)
Val images:  (2403, 640, 400)
Val labels:  (2403, 640, 400)
Test images:  (1440, 640, 400)


In [ ]:
# BATCH_SIZE = 8

# train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(BATCH_SIZE)
# val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(BATCH_SIZE)

In [14]:
image, masks_true = train_images[0], train_labels[0]
# tried image resizing to see if that fixes prediction shape broadcasting error, but doesn't seem to be it
# IMAGE_SIZE = (256,320)
# image = cv2.resize(image,IMAGE_SIZE,interpolation = cv2.INTER_LANCZOS4)
instances = model.detect([image], verbose=1)
instance = instances[0]
class_names=["BG", "sclera", "iris", "pupil"]
utils.display_instances(image, instance['rois'], instance['masks'], instance['class_ids'], class_names, instance['scores'])

Processing 1 images
image                    shape: (640, 400)            min:    0.00000  max:  255.00000  uint8


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (3,2) and requested shape (2,2)

In [ ]:
def compute_precision(gt_masks, pd_masks, threshold):
    """ compute the precision for instance segmentation
    
        Input:
            gt_masks (numpy array of shape (number of masks, IMAGE_SIZE)): ground truth masks
            pd_masks (numpy array of shape (IMAGE_SIZE, number of predictions)): predicted masks
            threshold (float): threshold for intersection over union of ground truth and predicted masks
        Output:
            precision (float): precision of instance segmentation task
    """ 
    
    true_pos = 0
    num_pos = pd_masks.shape[2]
    
    for i in range(num_pos):
        pd_mask = pd_masks[:,:,i]
        max_overlap = -1
        index_overlap = -1
        for j in range(len(gt_masks)):
            gt_mask = gt_masks[j][:,:,0]
            gt_mask = gt_mask.astype(int)
            gt_mask[gt_mask == 255] = True
            gt_mask[gt_mask == 0] = False
            curr_overlap = np.sum(np.equal(pd_mask.flatten(), gt_mask.flatten()))
            if curr_overlap > max_overlap:
                max_overlap = curr_overlap
                index_overlap = j
        matching_gt_mask = gt_masks[index_overlap][:,:,0]
        matching_gt_mask = matching_gt_mask.astype(int)
        matching_gt_mask[matching_gt_mask == 255] = True
        matching_gt_mask[matching_gt_mask == 0] = False
        
        x = matching_gt_mask.flatten()
        y = pd_mask.flatten()
        
        iou_num = np.sum(np.bitwise_and(x,y))
        iou_denom = np.sum(x) + np.sum(y) - np.sum(np.bitwise_and(x,y))
        iou = iou_num/iou_denom
        
        if iou >= threshold:
            true_pos += 1
        
    precision = true_pos/num_pos
    
    return precision

In [ ]:
precision_all = []
threshold = 0.5
for i in len(val_images):
    image = val_images[i]
    gt_masks = val_labels[i]
    pd_instances = model.detect([image])
    pd_masks = pd_instances[0]['masks']
    precision = compute_precision(gt_masks, pd_masks, threshold)
    precision_all.append(precision)

val_precision = np.mean(np.array(precision_all))
print("Validation precision: " + str(val_precision))